# Batch Scoring on IBM Cloud Pak for Data (ICP4D)

We are going to use this notebook to create and/or run a batch scoring job against a model that has previously been created and deployed to the Watson Machine Learning (WML) instance on Cloud Pak for Data (CP4D).

## 1.0 Install required packages


There are a couple of Python packages we will use in this notebook. First we make sure the Watson Machine Learning client v3 is removed (its not installed by default) and then install/upgrade the v4 version of the client (this package is installed by default on CP4D).
- WML Client: https://wml-api-pyclient-dev-v4.mybluemix.net/#repository

In [1]:
!pip uninstall watson-machine-learning-client -y
!pip install --user watson-machine-learning-client-v4==1.0.95 --upgrade | tail -n 1

In [2]:
import json
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## 2.0 Create Batch Deployment Job

### 2.1 Instantiate Watson Machine Learning Client

To interact with the local Watson Machine Learning instance, we will be using the Python SDK. 

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

<font color='red'>Replace the `username` and `password` values of `************` with your Cloud Pak for Data `username` and `password`. The value for `url` should match the `url` for your Cloud Pak for Data cluster, which you can get from the browser address bar (be sure to include the 'https://'.</font> The credentials should look something like this (these are example values, not the ones you will use):

`
wml_credentials = {
                   "url": "https://zen.clusterid.us-south.containers.appdomain.cloud",
                   "username": "cp4duser",
                   "password" : "cp4dpass",
                   "instance_id": "wml_local",
                   "version" : "2.5.0"
                  }
`
#### NOTE: Make sure that there is no trailing forward slash `/` in the `url`

In [3]:
# Be sure to update these credentials before running the cell.
wml_credentials = {
                   "url": "https://zen-cpd-zen.omid-cp4d-v5-2bef1f4b4097001da9502000c44fc2b2-0001.us-south.containers.appdomain.cloud",
                   "username": "<redacted>",
                   "password" : "<redacted>",
                   "instance_id": "wml_local",
                   "version" : "2.5.0"
                  }

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
wml_client.spaces.list()

------------------------------------  ----------------------  ------------------------
GUID                                  NAME                    CREATED
2fd56c70-0907-445c-9957-62ce5b1e1065  ScottDAdeploymentSpace  2020-06-17T17:05:07.749Z
d1c17f19-7b7c-4083-847e-7b0dbcee7fab  blart deployment space  2020-04-16T22:54:41.604Z
------------------------------------  ----------------------  ------------------------


### 2.2 Find Deployment Space

We will try to find the `GUID` for the deployment space you want to use and set it as the default space for the client.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update with the name of the deployment space where you have created the batch deployment.

In [5]:
# Be sure to update the name of the space with the one you want to use.
DEPLOYMENT_SPACE_NAME = 'ScottDAdeploymentSpace'

In [6]:
all_spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in all_spaces:
    if space['entity']['name'] == DEPLOYMENT_SPACE_NAME:
        space_id = space["metadata"]["guid"]
        print("\nDeployment Space GUID: ", space_id)

if space_id is None:
    print("WARNING: Your space does not exist. Create a deployment space before proceeding.")
    # We could programmatically create the space.
    #space_id = wml_client.spaces.store(meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]


Deployment Space GUID:  2fd56c70-0907-445c-9957-62ce5b1e1065


In [7]:
# Now set the default space to the GUID for your deployment space. If this is successful, you will see a 'SUCCESS' message.
wml_client.set.default_space(space_id)

'SUCCESS'

In [8]:
# These are the models and deployments we currently have in our deployment space.
wml_client.repository.list_models()
wml_client.deployments.list()

------------------------------------  ---------------------------------------------------------------------  ------------------------  --------------
GUID                                  NAME                                                                   CREATED                   TYPE
034e3195-2c5d-47de-8968-a9a2cabf822a  credit-risk autoAI camelCase - P4 GradientBoostingClassifierEstimator  2020-07-14T16:45:53.002Z  wml-hybrid_0.1
0d4c3282-d2c6-44aa-b9a3-e3525550da98  scottda model 7-14-2020                                                2020-07-14T12:56:00.002Z  mllib_2.3
f8f986d0-62b0-49e7-b42c-5745e7bd4a32  scottda model 7-8-2020 nocaps                                          2020-07-08T12:33:58.002Z  mllib_2.3
ac113f95-814f-413c-b276-3f8002f4ad7e  scottda model 6-25                                                     2020-06-25T14:54:23.002Z  mllib_2.3
1859b11e-7443-4918-9737-b8f8228c20c8  sda-model-6-17-2020                                                    2020-06-17T17:16

### 2.3 Find Batch Deployment

We will try to find the batch deployment which was created.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update the variable with the name of the batch deployment you created previously.

In [9]:
DEPLOYMENT_NAME = 'credit-risk-batch-deployment-model-7-14-2020'

In [10]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
deployment_details = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        deployment_details = deployment
        #print(json.dumps(deployment_details, indent=3))
        break

print("Deployment id: {}".format(deployment_uid))
wml_client.deployments.get_details(deployment_uid)

Deployment id: 04e16923-70cc-4e08-8c65-3e32416ad41e


{'metadata': {'parent': {'href': ''},
  'guid': '04e16923-70cc-4e08-8c65-3e32416ad41e',
  'modified_at': '',
  'created_at': '2020-07-14T17:21:58.647Z',
  'href': '/v4/deployments/04e16923-70cc-4e08-8c65-3e32416ad41e'},
 'entity': {'name': 'credit-risk-batch-deployment-model-7-14-2020',
  'custom': {},
  'description': '',
  'compute': {'name': 'XS', 'nodes': 1},
  'batch': {'schedule': {}},
  'space': {'href': '/v4/spaces/2fd56c70-0907-445c-9957-62ce5b1e1065'},
  'status': {'state': 'ready'},
  'asset': {'href': '/v4/models/0d4c3282-d2c6-44aa-b9a3-e3525550da98?space_id=2fd56c70-0907-445c-9957-62ce5b1e1065'},
  'auto_redeploy': False}}

### 2.4 Get Batch Test Data

We will load some data to run the batch predictions.

In [11]:
import pandas as pd

from project_lib import Project
project = Project.access()

batch_set = pd.read_csv(project.get_file('German-Credit-Risk-SmallBatchSet.csv'))
batch_set = batch_set.drop('CUSTOMERID', axis=1,errors = 'ignore')
batch_set.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,CurrentResidenceDuration,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker
0,0_to_200,4,all_credits_paid_back,car_new,250,100_to_500,less_1,2,female,none,3,real_estate,26,bank,rent,1,unskilled,1,none,yes
1,0_to_200,14,credits_paid_to_date,car_new,3148,less_100,1_to_4,3,male,none,3,car_other,41,none,own,2,skilled,1,none,yes
2,greater_200,19,credits_paid_to_date,radio_tv,5351,100_to_500,greater_7,4,male,none,3,savings_insurance,49,none,own,2,skilled,1,yes,yes
3,greater_200,34,outstanding_credit,other,5790,500_to_1000,greater_7,5,male,none,4,car_other,44,stores,own,2,unskilled,1,yes,yes
4,less_0,4,all_credits_paid_back,car_new,250,100_to_500,less_1,1,female,none,1,real_estate,21,bank,rent,1,unskilled,1,none,yes


### 2.5 Create Job

We can now use the information about the deployment and the test data to create a new job against our batch deployment. We submit the data as inline payload and want the results (i.e predictions) stored in a CSV file.

In [12]:
import time
timestr = time.strftime("%Y%m%d_%H%M%S")
job_payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': batch_set.columns.values.tolist(),
        'values': batch_set.values.tolist()
    }],
    wml_client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
            "type": "data_asset",
            "connection": {},
            "location": {
                "name": "batchres_{}_{}.csv".format(timestr,deployment_uid),
                "description": "results"
            }
    }
}

job = wml_client.deployments.create_job(deployment_id=deployment_uid, meta_props=job_payload)
job_uid = wml_client.deployments.get_job_uid(job)

print('Job uid = {}'.format(job_uid))

Job uid = 1325f900-d0d6-4800-84f5-e6c6ca1810f9


In [13]:
wml_client.deployments.list_jobs()

------------------------------------  ------  ------------------------  ------------------------------------
JOB-UID                               STATE   CREATED                   DEPLOYMENT-ID
1325f900-d0d6-4800-84f5-e6c6ca1810f9  queued  2020-07-14T17:29:17.507Z  04e16923-70cc-4e08-8c65-3e32416ad41e
------------------------------------  ------  ------------------------  ------------------------------------


## 3.0 Monitor Batch Job Status

The batch job is an async operation. We can use the identifier to track its progress. Below we will just poll until the job completes (or fails).

In [14]:
def poll_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(5)
            
job_details = poll_async_job(wml_client, job_uid)

{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'running', 'running_at': '2020-07-14T17:30:11.714Z', 'completed_at': ''}
{'state': 'completed', 'running_at': '', 'completed_at': ''}


In [15]:
wml_client.deployments.list_jobs()

------------------------------------  ---------  ------------------------  ------------------------------------
JOB-UID                               STATE      CREATED                   DEPLOYMENT-ID
1325f900-d0d6-4800-84f5-e6c6ca1810f9  completed  2020-07-14T17:29:17.507Z  04e16923-70cc-4e08-8c65-3e32416ad41e
------------------------------------  ---------  ------------------------  ------------------------------------


### 3.1 Check Results

With the job complete, we can see the predictions. 

In [16]:
wml_client.deployments.get_job_details()

{'resources': [{'metadata': {'guid': '1325f900-d0d6-4800-84f5-e6c6ca1810f9',
    'href': '/v4/deployment_jobs/1325f900-d0d6-4800-84f5-e6c6ca1810f9',
    'created_at': '2020-07-14T17:29:17.507Z',
    'parent': {'href': ''}},
   'entity': {'deployment': {'href': '/v4/deployments/04e16923-70cc-4e08-8c65-3e32416ad41e'},
    'scoring': {'input_data': [{'fields': ['CheckingStatus',
        'LoanDuration',
        'CreditHistory',
        'LoanPurpose',
        'LoanAmount',
        'ExistingSavings',
        'EmploymentDuration',
        'InstallmentPercent',
        'Sex',
        'OthersOnLoan',
        'CurrentResidenceDuration',
        'OwnsProperty',
        'Age',
        'InstallmentPlans',
        'Housing',
        'ExistingCreditsCount',
        'Job',
        'Dependents',
        'Telephone',
        'ForeignWorker'],
       'values': [['0_to_200',
         4,
         'all_credits_paid_back',
         'car_new',
         250,
         '100_to_500',
         'less_1',
         2

In [17]:
print(json.dumps(job_details, indent=2))

{
  "metadata": {
    "guid": "1325f900-d0d6-4800-84f5-e6c6ca1810f9",
    "href": "/v4/deployment_jobs/1325f900-d0d6-4800-84f5-e6c6ca1810f9",
    "created_at": "2020-07-14T17:29:17.507Z",
    "parent": {
      "href": ""
    }
  },
  "entity": {
    "deployment": {
      "href": "/v4/deployments/04e16923-70cc-4e08-8c65-3e32416ad41e"
    },
    "scoring": {
      "input_data": [
        {
          "fields": [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker"
          ],
          "v

## Congratulations, you have created and submitted a job for batch scoring !